# Import dei pacchetti

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import os
import keras as ks
from keras.preprocessing import image as kimage
import skimage.io as io
import random
import sys
import pickle as pkl
from imblearn.keras import BalancedBatchGenerator
from keras.utils import np_utils
lib_path = os.path.join(os.path.realpath(r"C:\Users\Giobi\OneDrive - Universita degli Studi di Milano-Bicocca\Digital Signal and Image Management\Progetto Segnali Stradali\DataAugmentationForObjectDetection-master"), "data_aug")
sys.path.append(lib_path)
from bbox_util import *
from data_aug import *
from imblearn.under_sampling import NearMiss
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.models import Model
from keras.layers import Concatenate,Input, Dense, Flatten, Reshape, MaxPooling2D, Conv2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dense, Dropout, BatchNormalization, LeakyReLU
from keras.optimizers import SGD
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, explained_variance_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils import shuffle
import skimage.measure
from scipy.spatial import distance
from collections import OrderedDict

# Funzioni utili

In [4]:
def find_only_one_signal(results, euclidean):
    is_sig = 0.6
    diz = {}
    conta = 0
    ob = np.squeeze(results)
    #print(type(ob))
    #print(ob.shape)
    if ob.shape == ():
        ob = np.array([ob])
    if (len(ob.shape)==1)&(results.shape[1]<=results.shape[2]):
        ob = np.expand_dims(ob, axis = 0)
    if (len(ob.shape)==1)&(results.shape[1]>results.shape[2]):
        ob = np.expand_dims(ob, axis = 1)
    #print(ob.shape)
    for i in range(0, ob.shape[0]):
        for j in range(0, ob.shape[1]):
            if ob[i][j] > 0.6:
                close = False
                for k in diz:
                    if (distance.euclidean([i,j], [diz[k]['i'],diz[k]['j']]) <= euclidean):
                        #print('troppo vicino: ' + str((i,j)) + ' da ' + str(((diz[k]['i']),(diz[k]['j']))), 'la distanza è: ' + str(distance.euclidean([i,j], [diz[k]['i'],diz[k]['j']])))
                        close = True
                        if (diz[k]['value']<ob[i][j]):
                            diz[k]['value'] = ob[i][j]
                            diz[k]['i'] = i
                            diz[k]['j'] = j           
                if (close == False):
                    d = {}
                    d['value'] = ob[i][j]
                    d['i'] = i
                    d['j'] = j
                    diz['n{}'.format(len(diz.keys()))] = d
            conta +=1
    out = np.zeros([ob.shape[0], ob.shape[1]])
    for k in diz:
        i = diz[k]['i']
        j = diz[k]['j']
        out[i][j] = diz[k]['value']
    return out

In [5]:
def crops_with_signal(image, results, show = False):
    diz = {}
    ob = np.squeeze(results)
    image = np.squeeze(image)
    if (len(ob.shape) == 1) & (image.shape[0]<=image.shape[1]):
        for j in range(0, len(ob)):
            if np.round(ob[j]) == 1:
                window = image[:40, 2*j:40 + 2*j,  :]
                d = {}
                d['j'] = j
                d['prob'] = ob[j]
                d['formula'] = 'image[:40, 2*j:40 + 2*j,  :]'
                d['array'] = window
                diz['im{}'.format(len(diz.keys()))] = d
                if show == True:
                    plt.imshow(window); plt.show()
                    
    if (len(ob.shape) == 1) & (image.shape[0]>image.shape[1]):
        for j in range(0, len(ob)):
            if np.round(ob[j]) == 1:
                window = image[2*j:40 + 2*j, :40, :]
                d = {}
                d['j'] = j
                d['prob'] = ob[j]
                d['formula'] = 'image[2*j:40 + 2*j, :40, :]'
                d['array'] = window
                diz['im{}'.format(len(diz.keys()))] = d
                if show == True:
                    plt.imshow(window); plt.show()
                    
    if (len(ob.shape) > 1):
        for i in range(0, ob.shape[0]):
            for j in range(0, ob.shape[1]):
                if (np.round(ob[i][j]) == 1):
                    window = image[2*i : 40 + 2*i, 2*j:40 + 2*j,  :]
                    d = {}
                    d['j'] = j
                    d['i'] = i
                    d['prob'] = ob[i][j]
                    d['formula'] = 'image[2*i : 40 + 2*i, 2*j:40 + 2*j,  :]'
                    d['array'] = window
                    diz['im{}'.format(len(diz.keys()))] = d
                    if show == True:
                        plt.imshow(window); plt.show()
    return diz

In [6]:
def resize_img(im):
    multipli40 = np.arange(start = 40, stop = max(im.shape) + 40, step = 40)
    index_x = np.argmin(np.absolute(multipli40-im.shape[1]))
    new_x = multipli40[index_x]
    index_y = np.argmin(np.absolute(multipli40-im.shape[0]))
    new_y = multipli40[index_y]
    resized = cv2.resize(im, (new_x, new_y), interpolation = cv2.INTER_AREA)
    return resized

In [7]:
def reshape_img(im):
    im = np.expand_dims(im, axis = 0)
    r = 0
    c = 0
    new_im = im[0, 0:40,0:40]
    new_im = np.expand_dims(new_im, axis = 0)
    for i in range(0, int(im.shape[1]/40)):
        for j in range(0, int(im.shape[2]/40)):
            if ((i==0)&(j==0)):
                new_im = new_im
            else:
                app = np.expand_dims(im[0, r:r+40, c:c+40], axis = 0)
                new_im = np.concatenate([new_im, app], axis = 0)
            c = c + 40
        r = r + 40
        c = 0
    return new_im

In [8]:
def try_new_size(im, new_x, new_y):
    new_im =  cv2.resize(im, (new_x, new_y), interpolation = cv2.INTER_AREA)
    plt.imshow(new_im); plt.show()
    new_im = reshape_img(new_im)
    return new_im

In [9]:
def multi_task_prediction(diz):
    for i in diz:
        signal, bbox = model_multi_task.predict(np.expand_dims(diz[i]['array'], axis = 0))
        signal = np.argmax(signal)
        diz[i]['signal'] = df[df['target']==signal]['title']
        diz[i]['bbox'] = bbox
    return diz

In [10]:
def new_draw_rect(img, bboxes):
    x1 = bboxes[0]
    y1 = bboxes[1]
    x2 = bboxes[2]
    y2 = bboxes[3]
    new_bb = np.array([[x1, y1, x2, y2, 0]])
    plt.imshow(draw_rect(img, new_bb)); plt.show()

In [11]:
def show_results(diz, im, probability = False):    
    for i in diz:
        if i!='shape':
            if (probability == True):
                print("L'immagine contiene il segnale di {}".format(diz[i]['signal'].to_string()) + ". Il segnale è presente nella bounding box con probabilità pari a {}".format(diz[i]['prob']))      
                new_draw_rect(im, np.squeeze(diz[i]['new_bbox_coords']))
            else:
                print("L'immagine contiene il segnale di {}".format(diz[i]['signal'].to_string()))
                new_draw_rect(im, np.squeeze(diz[i]['new_bbox_coords']))

In [12]:
def full_image_with_bboxes(img, diz):
    image = np.squeeze(img).copy()
    if len(diz)>0: #se il dizionario non è vuoto
        formula = diz[list(diz.keys())[0]]['formula']
        if formula == 'image[:40, 2*j:40 + 2*j,  :]':
            for i in diz:
                if i!='shape':
                    j = diz[i]['j']
                    image[:40, 2*j:40 + 2*j,  :] = draw_rect(diz[i]['array'], np.concatenate([diz[i]['bbox'], np.array([[0]])], axis = 1))

        if formula == 'image[2*j:40 + 2*j, :40, :]':
            for i in diz:
                if i!='shape':
                    j = diz[i]['j']
                    image[2*j:40 + 2*j, :40, :] = draw_rect(diz[i]['array'], np.concatenate([diz[i]['bbox'], np.array([[0]])], axis = 1))

        if formula == 'image[2*i : 40 + 2*i, 2*j:40 + 2*j,  :]':
            for i in diz:
                if i!='shape':
                    k = diz[i]['i']
                    j = diz[i]['j']
                    image[2*k : 40 + 2*k, 2*j:40 + 2*j,  :] = draw_rect(diz[i]['array'], np.concatenate([diz[i]['bbox'], np.array([[0]])], axis = 1))

    return image

In [13]:
def resize_img(im):
    multipli40 = np.arange(start = 40, stop = max(im.shape) + 40, step = 40)
    index_x = np.argmin(np.absolute(multipli40-im.shape[1]))
    new_x = multipli40[index_x]
    index_y = np.argmin(np.absolute(multipli40-im.shape[0]))
    new_y = multipli40[index_y]
    resized = cv2.resize(im, (new_x, new_y), interpolation = cv2.INTER_AREA)
    return resized

In [14]:
def try_new_size(im, new_x, new_y, show= False):
    new_im =  cv2.resize(im, (new_y, new_x), interpolation = cv2.INTER_AREA)
    if show ==True:
        plt.imshow(new_im); plt.show()
    #new_im = reshape_img(new_im)
    return new_im

In [15]:
def add_bbox_coords(diz):
    if len(diz)>0:
        formula = diz[list(diz.keys())[0]]['formula']
        if (formula == 'image[2*i : 40 + 2*i, 2*j:40 + 2*j,  :]'):
            for i in diz:
                j = diz[i]['j']
                k = diz[i]['i']
                old_window = np.array([[0,0,40,40]])
                new_window= np.array([[2*j, 2*k, 40 + 2*j, 40 + 2*k]])
                difference = new_window - old_window
                old_bbox_coords = diz[i]['bbox']
                diz[i]['new_bbox_coords'] = old_bbox_coords + difference
        if (formula == 'image[:40, 2*j:40 + 2*j,  :]'):
            for i in diz:
                j = diz[i]['j']
                old_window = np.array([[0,0,40,40]])
                new_window= np.array([[2*j, 0, 40 + 2*j, 40]])
                difference = new_window - old_window
                old_bbox_coords = diz[i]['bbox']
                diz[i]['new_bbox_coords'] = old_bbox_coords + difference
        if (formula == 'image[2*j:40 + 2*j, :40, :]'):
            for i in diz:
                j = diz[i]['j']
                old_window = np.array([[0,0,40,40]])
                new_window= np.array([[0, 2*j, 40, 40 + 2*j]])
                difference = new_window - old_window
                old_bbox_coords = diz[i]['bbox']
                diz[i]['new_bbox_coords'] = old_bbox_coords + difference
    return diz

In [16]:
def create_other_diz(img, size, euclidean):
    img = np.squeeze(img).copy()
    img = try_new_size(img, int(size[0]),int(size[1]))
    shape = img.shape
    img = np.expand_dims(img, axis = 0)
    res_img = model_conv.predict(img)
    #print(res_img)
    diz = find_only_one_signal(res_img, euclidean)
    diz = crops_with_signal(img, diz)
    diz = multi_task_prediction(diz)
    diz = add_bbox_coords(diz)
    #if int(size[0]) == 120:
        #plt.imshow(diz[list(diz.keys())[0]]['array']); plt.show()
    diz['shape'] = shape
    return diz

In [17]:
def generate_sizes(shape, iterations):
    first = False
    if shape[0]>shape[1]:
        first = True
    min_shape = min(shape)
    ratio = max(shape)/min(shape)
    difference_from_40 = min_shape - 40
    list_of_shapes = []
    for i in range(1, iterations + 1, 1):
        if first == True:
            list_of_shapes.append(((min_shape-difference_from_40)*i*ratio, (min_shape-difference_from_40)*i))
        else:
            list_of_shapes.append(((min_shape-difference_from_40)*i, (min_shape-difference_from_40)*i*ratio))
    return list_of_shapes

In [18]:
def bbox_control(original_diz, list_of_dict, euclidean):
    new_diz = {}
    original_shape = original_diz['shape']
    for diz in list_of_dict:
        if len(diz)>1:
            for im in diz:
                if (im!='shape'):
                    ratiox = original_shape[0]/diz['shape'][0]
                    ratioy = original_shape[1]/diz['shape'][1]
                    x1 = np.squeeze(diz[im]['new_bbox_coords'])[0]*ratiox
                    y1 = np.squeeze(diz[im]['new_bbox_coords'])[1]*ratioy
                    x2 = np.squeeze(diz[im]['new_bbox_coords'])[2]*ratiox
                    y2 = np.squeeze(diz[im]['new_bbox_coords'])[3]*ratioy
                    bbox = np.array([[x1, y1, x2, y2]])
                    min_dist = 10000
                    dist = 10000
                    found = False
                    if len(original_diz) == 1:
                        original_diz['im0'] = diz[im]
                        original_diz['im0']['new_bbox_coords'] = bbox
                        appoggio = original_diz.copy()
                        original_diz = OrderedDict((k, appoggio[k]) for k in ['im0', 'shape'])
                    #plt.imshow(diz[im]['array']); plt.show()
                    for item in original_diz:
                        if item!='shape':
                            dist = distance.euclidean(original_diz[item]['new_bbox_coords'], bbox) #distanza tra le coordinate del dizionario originale e dell'item     
                            #if diz['shape']==120:
                                #print(dist)
                            if dist<min_dist:
                                min_dist = dist
                                immagine = item
                                found = True #è stata trovata una bbox nel dizionario corrente che contiene immagini sottoposte a resize   
                    if found == True:
                        if (min_dist <= euclidean)&(diz[im]['prob']>original_diz[immagine]['prob']):
                            original_diz[immagine]= diz[im]
                            original_diz[immagine]['new_bbox_coords'] = bbox
                        if (min_dist>euclidean):
                            key = 'im{}'.format(len(original_diz))
                            original_diz[key] = diz[im]
                            original_diz[key]['new_bbox_coords'] = bbox

    return original_diz

In [19]:
def clean_dict(diz, euclidean):
    if len(diz)>1:
        new_d = {}
        new_d[list(diz.keys())[0]] = {'prob':diz[list(diz.keys())[0]]['prob'], 'new_bbox_coords' : diz[list(diz.keys())[0]]['new_bbox_coords'], 'signal': diz[list(diz.keys())[0]]['signal']}       
        for i in diz:
            if i!='shape':
                dist = 10000
                min_dist = 10000
                found = False
                for item in new_d:
                    dist = distance.euclidean(new_d[item]['new_bbox_coords'], diz[i]['new_bbox_coords']) #distanza tra le coordinate del dizionario originale e dell'item     
                    if dist<min_dist:
                        min_dist = dist
                        immagine = item
                        found = True        
                if found == True:
                    if (min_dist <= euclidean)&(diz[i]['prob']>new_d[immagine]['prob']):
                        key = immagine
                        d = {'prob': diz[i]['prob'],
                            'signal': diz[i]['signal'],
                            'new_bbox_coords': diz[i]['new_bbox_coords']}
                        new_d[key] = d
                    if (min_dist>euclidean):
                        key = 'im{}'.format(len(new_d))
                        d = {'prob': diz[i]['prob'],
                            'signal': diz[i]['signal'],
                            'new_bbox_coords': diz[i]['new_bbox_coords']}
                        new_d[key] = d
        return new_d
    else:
        return diz

In [20]:
def execution(img, iterations = 12, return_shapes = False, euclidean = 15):
    img = resize_img(img)
    original_shape = img.shape
    img = np.expand_dims(img, axis = 0)
    res_img = model_conv.predict(img)
    diz = find_only_one_signal(res_img, euclidean)
    diz = crops_with_signal(img, diz)
    diz = multi_task_prediction(diz)
    diz = add_bbox_coords(diz)
    diz['shape'] = original_shape
    list_of_dict = []
    list_of_shapes = generate_sizes(original_shape[0:2], iterations)
    for i in list_of_shapes:
        list_of_dict.append(create_other_diz(img, i, euclidean))
        #print(len(list_of_shapes))
    #controllo le bbox 
    new_d = bbox_control(diz, list_of_dict, euclidean)
    new_d = clean_dict(new_d, euclidean)
    if return_shapes == False:
        return new_d, np.squeeze(img)
    else:
        return new_d, np.squeeze(img), list_of_shapes

# Import dei dati & dei modelli

In [22]:
X_1 = np.load(r'C:\Users\Giobi\OneDrive - Universita degli Studi di Milano-Bicocca\Digital Signal and Image Management\Progetto Segnali Stradali\array_da_importare/train_speriamo_bene_completo_X.npy')
y_1 = np.load(r'C:\Users\Giobi\OneDrive - Universita degli Studi di Milano-Bicocca\Digital Signal and Image Management\Progetto Segnali Stradali\array_da_importare/train_speriamo_bene_completo_Y.npy')
X_train1, y_train1 = X_1, y_1
X_train1 = X_train1/255
df = pd.read_csv(r'C:\Users\Giobi\OneDrive - Universita degli Studi di Milano-Bicocca\Digital Signal and Image Management\Progetto Segnali Stradali\final_segnali_stradali.csv', sep = ';')
X_test = np.load(r'C:\Users\Giobi\OneDrive - Universita degli Studi di Milano-Bicocca\Digital Signal and Image Management\Progetto Segnali Stradali\X_test.npy')
y_test = np.load(r'C:\Users\Giobi\OneDrive - Universita degli Studi di Milano-Bicocca\Digital Signal and Image Management\Progetto Segnali Stradali\y_test.npy')
X_test = X_test/255

In [35]:
def model_conv():
    model = Sequential()
    model.add(Conv2D(filters=8, kernel_size=7, strides=1, input_shape=(None,None,3))) #(None,None,3)
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16, kernel_size=9, strides=1))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2, strides=2))
    model.add(Conv2D(filters=32, kernel_size=9, strides=1))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 64, kernel_size=5, strides=1))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 1, kernel_size=1, strides=1))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(Activation('sigmoid'))
    return model

In [36]:
def model_multi_task():
    visible = Input(shape=(40,40,3)) #(None,None,3)
    x1 = Conv2D(16, kernel_size=3, strides=1)(visible)
    x1 = LeakyReLU(alpha=0.1)(x1)
    x1 = BatchNormalization()(x1)
    x2 = Conv2D(32, kernel_size=3, strides=1)(x1)
    x2 = LeakyReLU(alpha=0.1)(x2)
    x2 = BatchNormalization()(x2)
    x2 = MaxPooling2D(pool_size=(2, 2))(x2)
    x3 = Conv2D(32, kernel_size=3, strides=1)(x2)
    x3 = LeakyReLU(alpha=0.1)(x3)
    x3 = BatchNormalization()(x3)
    x4 = Conv2D(43, kernel_size=3, strides=1)(x3)
    x4 = LeakyReLU(alpha=0.1)(x4)
    x4 = BatchNormalization()(x4)
    x4 = MaxPooling2D(pool_size=(2, 2))(x4)
    x5 = Conv2D(43, kernel_size=1, strides=2)(x4)
    x5 = LeakyReLU(alpha=0.1)(x5)
    x5 = BatchNormalization()(x5)
    x_f = Flatten()(x5)
    x_1 = Dense(512, activation = 'relu')(x_f)
    x_1 = Dropout(0.5)(x_1)
    output1 = Dense(43, activation = 'softmax', name = 'classificazione') (x_1)
    x_2 = Dense(512, activation = 'relu')(x_f)
    x_2 = Dropout(0.5)(x_2)
    output2 = Dense(4, activation='linear', name = 'regressione')(x_2)
    model = Model(inputs=visible, outputs=[output1, output2])
    return model

In [37]:
model_conv = model_conv()
model_conv.load_weights(r'C:\Users\Giobi\OneDrive - Universita degli Studi di Milano-Bicocca\Digital Signal and Image Management\Progetto Segnali Stradali\weights\model_is_signal_conv.hdf5')
model_multi_task = model_multi_task()
model_multi_task.load_weights(r'C:\Users\Giobi\OneDrive - Universita degli Studi di Milano-Bicocca\Digital Signal and Image Management\Progetto Segnali Stradali\weights\classificazione_multiclasse.hdf5')